## 1. Configuração e Importações

In [9]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 80)

# Configuração de caminhos
INPUT_DIR = '../Data Layer/raw/'
OUTPUT_DIR = '../Data Layer/silver/'
OUTPUT_FILE_CSV = 'tb_partidas_completa.csv'

# Configuração do banco de dados PostgreSQL
DB_CONFIG = {
    'host': 'localhost',
    'port': 5433,
    'database': 'brasileirao',
    'user': 'postgres',
    'password': 'postgres'
}

# String de conexão
CONNECTION_STRING = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(CONNECTION_STRING, pool_size=10, max_overflow=20, echo=False)

print("Configuração carregada")
print(f"Input: {INPUT_DIR}")
print(f"Output: {OUTPUT_DIR}")
print(f"Database: {DB_CONFIG['database']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}")

Configuração carregada
Input: ../Data Layer/raw/
Output: ../Data Layer/silver/
Database: brasileirao@localhost:5433


## 2. Funções Auxiliares


Este módulo contém funções especializadas para transformação de dados esportivos, garantindo qualidade e consistência nas análises. Cada função foi desenhada para resolver problemas específicos identificados nos dados brutos do Campeonato Brasileiro:

- **Padronização**: Elimina inconsistências que impactam agregações e filtros
- **Enriquecimento**: Cria métricas derivadas que facilitam análises de negócio
- **Qualidade**: Trata valores nulos de forma contextual e inteligente

Essas transformações são fundamentais para:
- Análises táticas por comissões técnicas
- Relatórios de desempenho para dirigentes
- Insights para broadcasters e patrocinadores
- Modelos preditivos de resultados

### 2.1 Normalização de Strings

Nomes de times inconsistentes (ex: "Flamengo", " Flamengo ", "Flamengo  ") causam:
- **Duplicação de registros** em relatórios agregados
- **Erros em dashboards** que mostram o mesmo clube múltiplas vezes
- **Falhas em joins** ao cruzar dados de diferentes fontes

**Impacto Prático:**
- Rankings de times corretos e completos
- Receitas de patrocínio calculadas sem duplicação
- Relatórios de performance confiáveis para gestores

In [10]:
def normalize_string(s):
    """
    Remove espaços extras de strings.
    
    Exemplos:
        '  Flamengo  ' → 'Flamengo'
        'São Paulo ' → 'São Paulo'
    """
    if isinstance(s, str):
        return s.strip()
    return s

def normalize_dataframe_strings(df):
    """
    Aplica normalização em todas colunas string de um DataFrame.
    """
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(normalize_string)
    return df

print("Funções de normalização definidas")

Funções de normalização definidas


### 2.2 Conversão de Percentuais

Estatísticas como **posse de bola** (45%) e **precisão de passes** (87.5%) chegam como texto e precisam ser convertidas para números.

**Por que isso é crítico?**
- **Análise Tática**: Comissões técnicas precisam calcular médias e comparar desempenho
- **Scouting**: Identificar times com alta posse de bola para planejamento de jogos
- **Mídia Esportiva**: Criar gráficos comparativos de eficiência técnica
- **Apostas**: Modelos estatísticos dependem de valores numéricos precisos

**Exemplo de Aplicação:**
- Times com >55% de posse geralmente são dominantes
- Precisão de passes >80% indica time tecnicamente qualificado

In [11]:
def convert_percentage_to_float(value):
    """
    Converte strings percentuais em números decimais.
    
    Exemplos:
        '45%' → 45.0
        '87.5%' → 87.5
        NaN → NaN
    """
    if pd.isna(value) or value == '':
        return np.nan
    
    value_str = str(value).strip().rstrip('%')
    try:
        return float(value_str)
    except ValueError:
        return np.nan

def convert_percentage_columns(df, columns):
    """
    Aplica conversão de percentuais em colunas específicas.
    """
    for col in columns:
        if col in df.columns:
            df[col] = df[col].apply(convert_percentage_to_float)
    return df

print("Funções de conversão percentual definidas")

Funções de conversão percentual definidas


### 2.3 Criação de Colunas Derivadas

Dados brutos contêm apenas fatos básicos (data, placar). Criamos **métricas derivadas** que geram insights diretos:

**Colunas Temporais:**
- `ano`: Análise de evolução histórica de times
- `mes`: Identificar períodos de melhor/pior desempenho
- `dia_semana`: Otimizar programação de jogos para audiência

**Colunas de Resultado:**
- `tipo_resultado`: Classificar vitórias mandante/visitante/empate
- `diferenca_gols`: Medir margem de vitória (competitividade)
- `total_gols`: Identificar jogos de alto/baixo escore
- `foi_equilibrado`: Jogos decididos por 1 gol (máxima emoção)
- `foi_goleada`: Vitórias por 3+ gols (domínio absoluto)

**Aplicações de Negócio:**
- **Broadcasters**: Priorizar transmissão de jogos equilibrados (maior audiência)
- **Clubes**: Analisar vantagem de jogar em casa (% vitórias mandante)
- **Comissões Técnicas**: Identificar padrões de goleadas sofridas
- **Calendário**: Evitar jogos importantes em dias de baixa presença

In [12]:
def create_temporal_columns(df, date_column='data'):
    """
    Cria colunas temporais derivadas a partir de uma data.
    
    Adiciona: ano, mes, dia_semana
    """
    df['ano'] = df[date_column].dt.year
    df['mes'] = df[date_column].dt.month
    df['dia_semana'] = df[date_column].dt.day_name()
    return df

def create_result_columns(df):
    """
    Cria colunas relacionadas ao resultado da partida.
    
    Adiciona: tipo_resultado, diferenca_gols, total_gols, foi_equilibrado, foi_goleada
    """
    df['tipo_resultado'] = df.apply(
        lambda row: 'Vitória Mandante' if row['vencedor'] == row['mandante']
        else ('Vitória Visitante' if row['vencedor'] == row['visitante'] else 'Empate'),
        axis=1
    )
    
    df['diferenca_gols'] = abs(df['mandante_placar'] - df['visitante_placar'])
    df['total_gols'] = df['mandante_placar'] + df['visitante_placar']
    df['foi_equilibrado'] = df['diferenca_gols'] <= 1
    df['foi_goleada'] = df['diferenca_gols'] >= 3
    
    return df

print("Funções de enriquecimento temporal e de resultado definidas")

Funções de enriquecimento temporal e de resultado definidas


### 2.4 Agregação de Estatísticas

Esta função consolida estatísticas detalhadas (chutes, passes, cartões) que estão dispersas em múltiplos registros, criando uma **visão única por partida**.

**Estatísticas Agregadas:**
1. **Chutes e Chutes no Alvo**: Medir poder ofensivo e eficiência
2. **Posse de Bola**: Indicador de domínio tático
3. **Passes e Precisão**: Qualidade técnica do time
4. **Faltas**: Intensidade física e estilo de jogo
5. **Cartões**: Nível de disciplina tática
6. **Impedimentos**: Sincronização ofensiva
7. **Escanteios**: Pressão territorial

**Valor Estratégico:**

**Para Comissões Técnicas:**
- Preparar estratégia contra adversários específicos
- Identificar pontos fracos do time (ex: baixa eficiência de finalização)

**Para Analistas de Performance:**
- Correlacionar posse de bola com vitórias
- Avaliar eficiência: chutes no alvo / total de chutes

**Para Scouting:**
- Identificar times com estilo de jogo compatível
- Avaliar disciplina tática (cartões amarelos/vermelhos)

**Exemplo de Insight:**
- Time com 60% posse + baixa conversão de chutes → problema tático ofensivo

In [13]:
def aggregate_team_stats(df_stats, df_partidas, team_type='mandante'):
    """
    Agrega estatísticas para mandante ou visitante.
    
    Args:
        df_stats: DataFrame com estatísticas
        df_partidas: DataFrame com dados das partidas
        team_type: 'mandante' ou 'visitante'
    
    Returns:
        DataFrame com estatísticas agregadas
    """
    stats_cols = ['chutes', 'chutes_no_alvo', 'posse_de_bola', 'passes', 
                  'precisao_passes', 'faltas', 'cartao_amarelo', 'cartao_vermelho', 
                  'impedimentos', 'escanteios']
    
    df_merged = df_stats.merge(
        df_partidas[['id', team_type]], 
        left_on='partida_id', 
        right_on='id', 
        how='inner'
    )
    
    df_filtered = df_merged[df_merged['clube'] == df_merged[team_type]]
    
    rename_dict = {col: f'{team_type}_{col}' for col in stats_cols if col in df_filtered.columns}
    df_filtered = df_filtered.rename(columns=rename_dict)
    
    output_cols = ['partida_id'] + [f'{team_type}_{col}' for col in stats_cols if col in df_stats.columns]
    df_output = df_filtered[output_cols]
    
    return df_output

print("Função de agregação de estatísticas definida")

Função de agregação de estatísticas definida


### 2.5 Agregação de Gols

**Análise de Momentos Decisivos:**

Gols são os eventos mais importantes no futebol. Esta função categoriza e conta tipos específicos de gols para análises avançadas.

**Métricas Geradas:**
1. **Gols Contra**: Indica fragilidade defensiva sob pressão
2. **Gols de Pênalti**: Avaliar eficiência de cobradores e árbitros
3. **Total de Gols Registrados**: Volume ofensivo por partida

**Insights de Negócio:**

**Para Gestão de Time:**
- Alto índice de gols contra → trabalho psicológico necessário
- Muitos pênaltis sofridos → rever posicionamento defensivo
- Baixo total de gols → investir em atacantes

**Para Análise de Arbitragem:**
- Distribuição de pênaltis por árbitro
- Times que mais/menos recebem pênaltis

**Para Broadcasters:**
- Identificar jogos com alta probabilidade de gols (maior emoção)
- Destacar momentos de pênalti (pico de audiência)

**Estatística Reveladora:**
- 5-7% dos gols são pênaltis (momentos de máxima tensão)
- 3-5% são gols contra (erros sob pressão)

In [14]:
def aggregate_goals(df_gols):
    """
    Agrega informações de gols por partida.
    
    Returns:
        DataFrame com: gols_contra, gols_penalty, total_gols_registrados
    """
    gols_agg = df_gols.groupby('partida_id').agg({
        'tipo_de_gol': lambda x: (x == 'Gol Contra').sum(),
        'atleta': 'count'
    }).rename(columns={'tipo_de_gol': 'gols_contra', 'atleta': 'total_gols_registrados'})
    
    penalty_count = df_gols[df_gols['tipo_de_gol'] == 'Penalty'].groupby('partida_id').size()
    gols_agg['gols_penalty'] = penalty_count
    gols_agg['gols_penalty'] = gols_agg['gols_penalty'].fillna(0).astype(int)
    
    return gols_agg

print("Função de agregação de gols definida")

Função de agregação de gols definida


### 2.6 Agregação de Cartões

**Indicadores de Disciplina e Comportamento Tático:**

Cartões amarelos e vermelhos são mais que simples punições - revelam **padrões estratégicos e psicológicos** dos times.

**Métricas de Disciplina:**
1. **Total de Cartões Amarelos**: Faltas táticas e nível de intensidade
2. **Total de Cartões Vermelhos**: Perda de controle emocional ou faltas graves
3. **Total de Cartões**: Indicador geral de disciplina

**Valor Estratégico:**

**Para Comissões Técnicas:**
- Identificar jogadores indisciplinados
- Evitar acúmulo de amarelos em jogos decisivos
- Preparar time para jogar com 10 (histórico de expulsões)

**Para Gestão de Desempenho:**
- Cartões vermelhos custam pontos: 70% dos times perdem com 10 jogadores
- Suspensões acumuladas impactam planejamento de elenco
- Alto índice de cartões pode indicar estresse psicológico

**Para Análise de Arbitragem:**
- Árbitros mais rigorosos vs. mais permissivos
- Padrões de cartões por estádio (pressão da torcida)

**Benchmarks de Mercado:**
- <2.0 cartões/jogo: time disciplinado
- 2.0-3.0: padrão normal
- 3.0: necessário trabalho comportamental

**Impacto Financeiro:**
- Expulsões em jogos importantes podem custar milhões em premiações perdidas

In [15]:
def aggregate_cards(df_cartoes):
    """
    Agrega informações de cartões por partida.
    
    Returns:
        DataFrame com: total_cartoes_amarelos, total_cartoes_vermelhos, total_cartoes
    """
    cartoes_agg = df_cartoes.groupby('partida_id').agg({
        'cartao': lambda x: (x == 'Amarelo').sum(),
        'atleta': 'count'
    }).rename(columns={'cartao': 'total_cartoes_amarelos', 'atleta': 'total_cartoes'})
    
    red_count = df_cartoes[df_cartoes['cartao'] == 'Vermelho'].groupby('partida_id').size()
    cartoes_agg['total_cartoes_vermelhos'] = red_count
    cartoes_agg['total_cartoes_vermelhos'] = cartoes_agg['total_cartoes_vermelhos'].fillna(0).astype(int)
    
    return cartoes_agg

print("Função de agregação de cartões definida")

Função de agregação de cartões definida


### 2.7 Tratamento de Valores Nulos

**Garantia de Qualidade de Dados para Decisões de Negócio:**

Valores nulos (NULLs) são um dos maiores desafios em análise de dados esportivos. Tratá-los incorretamente pode gerar **insights enganosos** e **decisões equivocadas**.

**Problema Identificado no Campeonato Brasileiro:**

Descobrimos um **padrão histórico** na coleta de dados:
- **2003-2014**: Apenas posse de bola e precisão de passes eram registrados
- **2015-2023**: Dados completos (chutes, passes, escanteios, etc.)
- **2024**: Retorno ao padrão antigo (dados limitados)

**Estratégia Inteligente de Tratamento:**

Diferenciamos dois tipos de "zero":
1. **Zero Real**: O evento não ocorreu (ex: 0 chutes no alvo)
2. **Zero por Ausência de Dados**: Informação não foi coletada

**Anos 2003-2014 (Dados Limitados)**:
- Chutes, passes detalhados → Converter para NULL (não coletados)
- Posse de bola, precisão → Manter valores reais

**Anos 2015-2023 (Dados Completos)**:
- Todos os zeros são reais (evento não aconteceu)
- Não converter para NULL

**Colunas Agregadas (Gols, Cartões)**:
- Sempre preencher com 0 quando NULL
- Garante integridade: partida sem gol = 0, não NULL

**Percentuais (Posse, Precisão)**:
- Strings vazias → NULL
- NULLs → Preencher com média (estimativa razoável)

1. **Filtrar por período**: Use apenas 2015-2023 para estatísticas avançadas
2. **Segmentar análises**: Separe períodos com diferentes níveis de coleta
3. **Documentar limitações**: Sempre informar contexto dos dados ao negócio

In [16]:
def fill_numeric_nulls(df, columns, fill_value=0):
    """
    Preenche valores nulos em colunas numéricas.
    """
    for col in columns:
        if col in df.columns and df[col].dtype in ['float64', 'int64']:
            df[col] = df[col].fillna(fill_value)
    return df

def fill_percentage_nulls_with_mean(df, columns):
    """
    Preenche valores nulos em percentuais com a média da coluna.
    """
    for col in columns:
        if col in df.columns and df[col].dtype in ['float64']:
            df[col] = df[col].fillna(df[col].mean())
    return df

def treat_missing_stats_by_year(df):
    """
    Trata valores faltantes considerando o padrão histórico dos dados.
    
    Padrão identificado:
    - 2003-2014: Apenas posse_de_bola e precisao_passes disponíveis
    - 2015-2023: Dados completos (chutes, passes, etc.)
    - 2024: Volta ao padrão antigo (apenas posse e precisão)
    
    Estratégia:
    - Valores 0 em anos sem dados: manter como NULL (informação não coletada)
    - Valores 0 em anos com dados: manter 0 (evento não ocorreu)
    - Strings vazias: converter para NULL
    """
    # Anos com dados completos de estatísticas
    ANOS_COM_DADOS_COMPLETOS = list(range(2015, 2024))
    
    # Colunas que só têm dados a partir de 2015
    colunas_modernas = ['chutes', 'chutes_no_alvo', 'passes', 
                        'mandante_chutes', 'mandante_chutes_no_alvo', 'mandante_passes',
                        'visitante_chutes', 'visitante_chutes_no_alvo', 'visitante_passes']
    
    for col in colunas_modernas:
        if col in df.columns:
            # Para anos sem dados completos, converter 0 em NULL
            mask = ~df['ano'].isin(ANOS_COM_DADOS_COMPLETOS) & (df[col] == 0)
            df.loc[mask, col] = np.nan
    
    # Tratar strings vazias em percentuais
    percent_cols = [col for col in df.columns if 'posse_de_bola' in col or 'precisao_passes' in col]
    for col in percent_cols:
        if col in df.columns:
            df[col] = df[col].replace('', np.nan)
    
    return df

print(" Funções de tratamento inteligente de valores nulos definidas")
print(" Pronto para garantir qualidade de dados em análises históricas")

 Funções de tratamento inteligente de valores nulos definidas
 Pronto para garantir qualidade de dados em análises históricas


## 3. Carregando os Dados Brutos (EXTRACT)

Leitura dos 4 datasets da camada RAW.

In [17]:
df_partidas = pd.read_csv(f'{INPUT_DIR}campeonato-brasileiro-full.csv')
df_gols = pd.read_csv(f'{INPUT_DIR}campeonato-brasileiro-gols.csv')
df_cartoes = pd.read_csv(f'{INPUT_DIR}campeonato-brasileiro-cartoes.csv')
df_estatisticas = pd.read_csv(f'{INPUT_DIR}campeonato-brasileiro-estatisticas-full.csv')

print("Dados brutos carregados com sucesso")
print(f"Partidas: {len(df_partidas):,} registros")
print(f"Gols: {len(df_gols):,} registros")
print(f"Cartões: {len(df_cartoes):,} registros")
print(f"Estatísticas: {len(df_estatisticas):,} registros")

Dados brutos carregados com sucesso
Partidas: 8,785 registros
Gols: 9,861 registros
Cartões: 20,953 registros
Estatísticas: 17,570 registros


## 4. Transformação dos Dados (TRANSFORM)

Pipeline modular de transformação aplicando funções auxiliares.

In [18]:
# Normalização de Strings
df_partidas = normalize_dataframe_strings(df_partidas)
df_gols = normalize_dataframe_strings(df_gols)
df_cartoes = normalize_dataframe_strings(df_cartoes)
df_estatisticas = normalize_dataframe_strings(df_estatisticas)

# Conversão de Percentuais
df_estatisticas = convert_percentage_columns(df_estatisticas, ['posse_de_bola', 'precisao_passes'])

# Preparação Base - Partidas
df = df_partidas.copy()
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y', errors='coerce')
df = create_temporal_columns(df, 'data')
df = create_result_columns(df)

cols_to_drop = ['formacao_mandante', 'formacao_visitante', 'tecnico_mandante', 'tecnico_visitante']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# Padronização de Estatísticas
df_estatisticas.columns = df_estatisticas.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')

# Agregação de Estatísticas
df_stats_mandante = aggregate_team_stats(df_estatisticas, df, 'mandante')
df_stats_visitante = aggregate_team_stats(df_estatisticas, df, 'visitante')

df = df.merge(df_stats_mandante, left_on='id', right_on='partida_id', how='left')
df = df.merge(df_stats_visitante, left_on='id', right_on='partida_id', how='left', suffixes=('', '_v'))

# Agregação de Gols
df_gols.columns = df_gols.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
gols_agg = aggregate_goals(df_gols)

df = df.merge(gols_agg, left_on='id', right_index=True, how='left')
df['gols_contra'] = df['gols_contra'].fillna(0).astype(int)
df['gols_penalty'] = df['gols_penalty'].fillna(0).astype(int)
df['total_gols_registrados'] = df['total_gols_registrados'].fillna(0).astype(int)

# Agregação de Cartões
df_cartoes.columns = df_cartoes.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
cartoes_agg = aggregate_cards(df_cartoes)

df = df.merge(cartoes_agg, left_on='id', right_index=True, how='left')
df['total_cartoes_amarelos'] = df['total_cartoes_amarelos'].fillna(0).astype(int)
df['total_cartoes_vermelhos'] = df['total_cartoes_vermelhos'].fillna(0).astype(int)
df['total_cartoes'] = df['total_cartoes'].fillna(0).astype(int)

# Limpeza Final
df = df.drop(columns=['partida_id', 'partida_id_v'], errors='ignore')
df = df.rename(columns={'id': 'partida_id'})

# Tratamento Inteligente de Nulos (considerando padrão histórico)
df = treat_missing_stats_by_year(df)

# Tratar valores remanescentes
stat_numeric_cols = [col for col in df.columns if any(x in col for x in ['chutes', 'passes', 'faltas', 'impedimentos', 'escanteios'])]
df = fill_numeric_nulls(df, stat_numeric_cols, fill_value=0)

percent_cols = [col for col in df.columns if 'posse_de_bola' in col or 'precisao_passes' in col]
df = fill_percentage_nulls_with_mean(df, percent_cols)

df_silver = df.copy()


## 5. Validação dos Dados

In [20]:
print(f"Registros transformados: {len(df_silver):,}")
print(f"Colunas finais: {len(df_silver.columns)}")
print(f"Times únicos: {df_silver['mandante'].nunique()}")
print(f"Período: {df_silver['ano'].min()} - {df_silver['ano'].max()}")

# Validação de tratamento de nulos
print("Análise de valores nulos após tratamento:")
# Agrupar por ano e verificar nulos em colunas críticas
cols_check = ['mandante_chutes', 'mandante_passes', 'mandante_posse_de_bola', 'total_gols_registrados']
for col in cols_check:
    if col in df_silver.columns:
        nulls_por_ano = df_silver.groupby('ano')[col].apply(lambda x: x.isna().sum())
        if nulls_por_ano.sum() > 0:
            print(f"\n{col}:")
            print(nulls_por_ano[nulls_por_ano > 0])
        else:
            print(f"{col}: Sem valores nulos")

df_silver.head()

Registros transformados: 8,785
Colunas finais: 46
Times únicos: 45
Período: 2003 - 2024
Análise de valores nulos após tratamento:
mandante_chutes: Sem valores nulos
mandante_passes: Sem valores nulos
mandante_posse_de_bola: Sem valores nulos
total_gols_registrados: Sem valores nulos


,partida_id,rodata,data,hora,mandante,visitante,vencedor,arena,mandante_placar,visitante_placar,mandante_estado,visitante_estado,ano,mes,dia_semana,tipo_resultado,diferenca_gols,total_gols,foi_equilibrado,foi_goleada,mandante_chutes,mandante_chutes_no_alvo,mandante_posse_de_bola,mandante_passes,mandante_precisao_passes,mandante_faltas,mandante_cartao_amarelo,mandante_cartao_vermelho,mandante_impedimentos,mandante_escanteios,visitante_chutes,visitante_chutes_no_alvo,visitante_posse_de_bola,visitante_passes,visitante_precisao_passes,visitante_faltas,visitante_cartao_amarelo,visitante_cartao_vermelho,visitante_impedimentos,visitante_escanteios,gols_contra,total_gols_registrados,gols_penalty,total_cartoes_amarelos,total_cartoes,total_cartoes_vermelhos
0,1,1,2003-03-29,16:00,Guarani,Vasco,Guarani,Brinco de Ouro,4,2,SP,RJ,2003,3,Saturday,Vitória Mandante,2,6,False,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,2003-03-29,16:00,Athletico-PR,Gremio,Athletico-PR,Arena da Baixada,2,0,PR,RS,2003,3,Saturday,Vitória Mandante,2,2,False,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
2,3,1,2003-03-30,16:00,Flamengo,Coritiba,-,Maracanã,1,1,RJ,PR,2003,3,Sunday,Empate,0,2,True,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
3,4,1,2003-03-30,16:00,Goias,Paysandu,-,Serra Dourada,2,2,GO,PA,2003,3,Sunday,Empate,0,4,True,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
4,5,1,2003-03-30,16:00,Internacional,Ponte Preta,-,Beira Rio,1,1,RS,SP,2003,3,Sunday,Empate,0,2,True,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0


## 6. Salvando Backup em CSV

In [21]:
output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILE_CSV)
df_silver.to_csv(output_path, index=False)

print(f"Backup CSV salvo: {output_path}")

Backup CSV salvo: ../Data Layer/silver/tb_partidas_completa.csv


## 7. Carregamento no PostgreSQL (LOAD)

In [25]:
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT version();"))
        version = result.fetchone()[0]
        print(f"Conectado ao PostgreSQL: {version[:50]}...")
except Exception as e:
    print(f"Erro na conexão: {e}")

Conectado ao PostgreSQL: PostgreSQL 15.15 on x86_64-pc-linux-musl, compiled...


In [24]:
try:
    df_silver.to_sql(
        'tb_partidas_completa', 
        engine, 
        schema='silver',
        if_exists='replace',
        index=False,
        method='multi',
        chunksize=1000
    )
    print("Dados carregados no PostgreSQL (schema SILVER)")
except Exception as e:
    print(f"Erro ao carregar dados: {e}")

Dados carregados no PostgreSQL (schema SILVER)


In [26]:
try:
    query = "SELECT COUNT(*) as total FROM silver.tb_partidas_completa"
    result = pd.read_sql(query, engine)
    print(f"Total de registros no banco: {result['total'][0]:,}")
    
    query_sample = "SELECT * FROM silver.tb_partidas_completa LIMIT 5"
    display(pd.read_sql(query_sample, engine))
except Exception as e:
    print(f"Erro: {e}")

Total de registros no banco: 8,785


,partida_id,rodata,data,hora,mandante,visitante,vencedor,arena,mandante_placar,visitante_placar,mandante_estado,visitante_estado,ano,mes,dia_semana,tipo_resultado,diferenca_gols,total_gols,foi_equilibrado,foi_goleada,mandante_chutes,mandante_chutes_no_alvo,mandante_posse_de_bola,mandante_passes,mandante_precisao_passes,mandante_faltas,mandante_cartao_amarelo,mandante_cartao_vermelho,mandante_impedimentos,mandante_escanteios,visitante_chutes,visitante_chutes_no_alvo,visitante_posse_de_bola,visitante_passes,visitante_precisao_passes,visitante_faltas,visitante_cartao_amarelo,visitante_cartao_vermelho,visitante_impedimentos,visitante_escanteios,gols_contra,total_gols_registrados,gols_penalty,total_cartoes_amarelos,total_cartoes,total_cartoes_vermelhos
0,1,1,2003-03-29,16:00,Guarani,Vasco,Guarani,Brinco de Ouro,4,2,SP,RJ,2003,3,Saturday,Vitória Mandante,2,6,False,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,2003-03-29,16:00,Athletico-PR,Gremio,Athletico-PR,Arena da Baixada,2,0,PR,RS,2003,3,Saturday,Vitória Mandante,2,2,False,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
2,3,1,2003-03-30,16:00,Flamengo,Coritiba,-,Maracanã,1,1,RJ,PR,2003,3,Sunday,Empate,0,2,True,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
3,4,1,2003-03-30,16:00,Goias,Paysandu,-,Serra Dourada,2,2,GO,PA,2003,3,Sunday,Empate,0,4,True,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
4,5,1,2003-03-30,16:00,Internacional,Ponte Preta,-,Beira Rio,1,1,RS,SP,2003,3,Sunday,Empate,0,2,True,False,0.0,0.0,51.698827,0.0,0.0,0,0,0,0,0,0.0,0.0,48.301173,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0
